# Import

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np 
from os import path
import pandas as pd 
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
import datetime
import math
import time
import pickle
import random
from scipy.spatial import cKDTree
from sklearn import preprocessing
from sklearn.decomposition import PCA
import sys
from sklearn.metrics import r2_score as r2
from rdkit import Chem
from sklearn.decomposition import PCA

/home/modules/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/modules/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# import importlib
# import efrc_ml_production as ml
# importlib.reload(ml)

<module 'efrc_ml_production' from '/home/rgur/py_scripts/efrc_ml_production.py'>

In [3]:
from skopt import gp_minimize

In [4]:
from multiprocessing import Pool

# Write Class

In [16]:
class hmofMLdataset:
    def __init__(self, results_dir, SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data.csv', 
                 SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data.csv',SI_stacked_path=
                '/data/rgur/efrc/prep_data/all_v1/stacked.csv',
                 SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked.csv',
                 Y_DATA_PATH='/data/rgur/efrc/data_DONOTTOUCH/hMOF_allData_March25_2013.xlsx', n_core=18):
        self.results_dir = results_dir
        os.chdir(self.results_dir)
        self.SI_grav_data_path = SI_grav_data_path
        self.SD_grav_data_path = SD_grav_data_path
        self.SI_stacked_path = SI_stacked_path
        self.SD_stacked_path = SD_stacked_path
        self.n_core = n_core
        self.Y_DATA_PATH = Y_DATA_PATH
        self.PCA_COMPONENTS = 400
        self.del_defective_mofs = False
        self.cat_si_sd = True
        self.add_size_fp = True #make True if you want to add 20 feature columns, where each feature is the number of atoms in a linker
        self.srt_size_fp = True
        self.iso_start_str_sd = 'Density'
        self.iso_end_str_sd = 'norm_Dom._Pore_(ang.)'
        self.grav_start_str_sd = 'CH4_v/v_248_bar'
        self.grav_end_str_sd = 'norm_Dom._Pore_(ang.)'
        self.start_str_si = 'filename'
        self.end_str_si = 'valence_pa'
        self.cat_col_names = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
        self.feature_codes = ['10000', '11000', '01000', '10100', '11100', '01100',
                             '10010', '11010', '01010', '10110', '11110', '01110',
                             '10001', '11001', '01001', '10101', '11101', '01101',
                             '10011', '11011', '01011', '10111', '11111', '01111']
        print("There are %s unique feature codes" %len(set(self.feature_codes)))
        print("now is %s" %now)
        
    
    def makeMasterDFs(self):
        #gravimetric
        self.grav, self.grav_prop, self.grav_target_mean, self.grav_target_std, self.grav_all_features = ml.prepToSplit(
                                            'xgb', self.cat_si_sd, self.SD_grav_data_path, self.SI_grav_data_path, 
                                            self.grav_start_str_sd, self.grav_end_str_sd, self.start_str_si, 
                                            self.end_str_si, 1, self.del_defective_mofs, self.add_size_fp, 
                                            self.srt_size_fp, None, stacked=False, n_core=self.n_core, 
                                            del_geometric_fp=False, cat_col_names=self.cat_col_names, 
                                            Y_DATA_PATH=self.Y_DATA_PATH)
        size_cols = ["size_%s" %s for s in range(20)]
        self.LS_dict = {row[1]['filename']:row[1][size_cols] for row in self.grav.iterrows()} # map from filename 
                                                                                        #to linkersize-vector
        #stacked
        self.iso, self.iso_prop, self.iso_target_mean, self.iso_target_std, self.iso_all_features, self.pinfo = \
                                            ml.prepToSplit(
                                            'nn', self.cat_si_sd, self.SD_stacked_path, self.SI_stacked_path, 
                                            self.iso_start_str_sd, self.iso_end_str_sd, self.start_str_si, 
                                            self.end_str_si, 1, self.del_defective_mofs, self.add_size_fp, 
                                            self.srt_size_fp, None, True, self.n_core, False, self.cat_col_names, 
                                            self.Y_DATA_PATH, self.LS_dict)

    def select_features(self, code, stacked):
        '''
        Should only be called after makeMasterDFs
        '''
        si = bool(int(code[0])) #True (=1) if size-independent features are included
        sd = bool(int(code[1])) #True (=1) if size-dependent features are included
        size_fp = bool(int(code[2])) #True (=1) if linker size features are included
        geo_fp = bool(int(code[3])) #True (=1) if geometric features are included
        non_pg = ml.getNonPGcolNames(size_fp, stacked, not geo_fp, self.cat_col_names)
        pg = []
        if si:
            si_df = pd.read_csv(self.SI_grav_data_path)
            self.all_pg = [s for s in ml.getPGcolNames(si_df, start_str=self.start_str_si, end_str=self.end_str_si)]
            pg += [s+'_si' for s in ml.getPGcolNames(si_df, start_str=self.start_str_si, end_str=self.end_str_si) 
                   if s+'_si' in self.grav_all_features]
            del si_df
        if sd:
            sd_df = pd.read_csv(self.SD_grav_data_path)
            pg += [s for s in ml.getPGcolNames(sd_df, self.grav_start_str_sd, self.grav_end_str_sd) if s in
                  self.grav_all_features]
        return non_pg + pg
    
    def makeResult(self, i):
        STACKED = bool(int(i[-1])) #True (=1) if stacked
        CODE = i[:-1]
        run_features = self.select_features(code=CODE, stacked=STACKED)
        if STACKED:
            print("Running code %s for isotherm model" %CODE)
            drop_features = [s for s in self.iso_all_features if s not in run_features]
            FpDataSet(self.iso.drop(drop_features, axis=1), run_features, self.iso_prop, 
                               self.iso_target_mean, self.iso_target_std, stacked=STACKED, fp_code=CODE).run()
        else:
            print("Running code %s for gravimetric uptake model" %CODE)
            drop_features = [s for s in self.grav_all_features if s not in run_features]
            FpDataSet(self.grav.drop(drop_features, axis=1), run_features, self.grav_prop, 
                               self.grav_target_mean, self.grav_target_std, stacked=STACKED, fp_code=CODE)        
    
    def makeAllResults(self):
        self.makeMasterDFs()
        print('\n')
        l = []
        for i in self.feature_codes: #True if stacked
                STACKED = bool(int(i[-1])) #True (=1) if stacked
                CODE = i[:-1]
                run_features = self.select_features(code=CODE, stacked=STACKED)
                if STACKED:
                    print("Running code %s for isotherm model" %CODE)
                    drop_features = [s for s in self.iso_all_features if s not in run_features]
                    #l.append(self.iso.drop(drop_features, axis=1))
                    FpDataSet(self.iso.drop(drop_features, axis=1), run_features, self.iso_prop, 
                                       self.iso_target_mean, self.iso_target_std, stacked=STACKED, fp_code=CODE).run()
                else:
                    print("Running code %s for gravimetric uptake model" %CODE)
                    drop_features = [s for s in self.grav_all_features if s not in run_features]
                    FpDataSet(self.grav.drop(drop_features, axis=1), run_features, self.grav_prop, 
                                       self.grav_target_mean, self.grav_target_std, stacked=STACKED, fp_code=CODE).run()
#         pool = Pool(1)
#         #feature_codes
#         result = pool.map(self.makeResult, self.feature_codes)
#         pool.close()
#         pool.join()

In [15]:
class FpDataSet:
    def __init__(self, df, features, property_used, target_mean, target_std, stacked, fp_code, PCA_DIM=400, 
                 rand_seeds=[0, 10, 20], train_grid = [.5, .6, .7, .8, .9]):
        self.df = df
        self.rand_seeds = rand_seeds
        self.fp_code = fp_code
        self.property_used = property_used
        self.target_mean = target_mean
        self.target_std = target_std
        self.n_samples = len(self.df)
        self.PCA_DIM = min(PCA_DIM, self.n_samples)
        self.features = features
        self.stacked = stacked
        if self.stacked:
            self.algo = 'nn'
            self.model_tag = 'iso'
        else:
            self.algo = 'xgb'
            self.model_tag = 'grav'
        self.train_grid = train_grid
    
    def sortRemoteInds(self):
        '''
        Create list of most remote indices
        '''
        n_core = 1
        use_pca = True
        nn_tree, filenames = ml.NNtree(self.df, self.stacked, n_core, self.features, use_pca, self.PCA_DIM)
        dt = nn_tree[0] #distance tree
        nt = nn_tree[1] #neighbor tree
        to_sort = list(zip(nt[:, 0],dt[:, 1]))

        srt_d = {}
        for i in to_sort:
            try:
                srt_d[i[0]] += i[1]
            except:
                srt_d[i[0]] = [i[1]]

        srt_ind_set = [(k, max(v)) for k,v in zip(srt_d.keys(), srt_d.values())]

        srt = sorted(srt_ind_set, key=lambda x: x[1], reverse=True)

        srt_inds = list([x[0] for x in srt])
        self.remote_info = [(x, filenames[x]) for x in srt_inds]
        
    def run(self):
        self.sortRemoteInds()
        #self.params_list = []
        #self.TTS = []
        self.results = []
#         TTS_obj =  trainTestSplit(self.df, .6, self.features, self.property_used,
#                                  self.target_mean, self.target_std, 0, self.remote_info,
#                                  self.stacked)
#        self.results_df, self.MODEL = TTS_obj.makeResults()
        
        for train_pct in self.train_grid:
            TRAIN_PCT = int(round(train_pct*100))
            for seed in self.rand_seeds:
#                 self.results.append(trainTestSplit(self.df, train_pct, self.features, self.property_used,
#                                  self.target_mean, self.target_std, seed, self.remote_info,
#                                  self.stacked)) 
#                 TTS_obj =  trainTestSplit(self.df, train_pct, self.features, self.property_used,
#                                                  self.target_mean, self.target_std, seed, self.remote_info,
#                                                  self.stacked)
#                 self.results_df, self.MODEL = TTS_obj.makeResults()
                results_df, MODEL = trainTestSplit(self.df, train_pct, self.features, self.property_used,
                                                  self.target_mean, self.target_std, seed, self.remote_info,
                                                  self.stacked).makeResults()
                save_fragment = '%s_code_%s_train_%s_seed_%s_%s' %(self.model_tag, self.fp_code, TRAIN_PCT, seed, now)
                print("Save Results using Fragment %s" %save_fragment)
                results_df.to_csv('results_%s.csv' %save_fragment)
                try:
                    MODEL.save_model('%s.xgb' %save_fragment)
                except:
                    MODEL.save('%s.h5' %save_fragment,save_format='h5')

In [8]:
class trainTestSplit:
    def __init__(self, df, train_pct, features, property_used, target_mean, target_std, seed, remote_info, stacked, 
                 hp_frac=.1, n_trees=5000):
        self.df = df
        self.filenames = df['filename'].unique().tolist()
        self.n_samples = len(self.filenames)
        self.train_pct = train_pct
        self.seed = seed
        self.target_mean = target_mean
        self.target_std = target_std
        self.remote_info = remote_info
        self.pct_remote = self.train_pct - .5
        self.n_remote = round(self.n_samples*self.pct_remote)
        self.n_train = round(self.n_samples*self.train_pct)
        self.n_random = self.n_train - self.n_remote
        self.stacked = stacked
        self.features = features
        self.property_used = property_used
        self.hp_frac = hp_frac
        self.n_trees = n_trees

        if self.stacked:
            self.algo = 'nn'
            self.model_tag = 'iso'
        else:
            self.algo = 'xgb'
            self.model_tag = 'grav'
        #print("algo is" %self.algo)
        
    
    def split(self):
        '''
        Split into train and test set
        '''

        self.train_fn = [x[1] for x in self.remote_info[:self.n_remote]]
        
        self.remaining = list(set(self.filenames) - set(self.train_fn))

        random.Random(self.seed).shuffle(self.remaining)

        self.train_fn += self.remaining[:self.n_random]

        self.test_fn = self.remaining[self.n_random:]
        train_df = self.df[self.df['filename'].isin(self.train_fn)].reset_index().drop('index', axis=1)
        test_df = self.df[self.df['filename'].isin(self.test_fn)].reset_index().drop('index', axis=1)
        print("Total len of test_df + train_df: %s" %(len(train_df) + len(test_df)))
        train_fp = train_df[self.features].to_numpy().astype('float32')
        train_label = train_df[self.property_used]
        test_fp = test_df[self.features].to_numpy().astype('float32')
        test_label = test_df[self.property_used]
    
        if self.algo == 'xgb':
            train_d = xgb.DMatrix(data=train_fp, label=train_label)
            test_d = xgb.DMatrix(data=test_fp, label=test_label)
            return train_d, test_d, train_label, test_label
        if self.algo == 'nn':
            train_files = train_df['filename'].tolist()
            test_files = test_df['filename'].tolist()
            train_pressures = train_df['pressure'].tolist()
            test_pressures = test_df['pressure'].tolist()
            return (train_fp, train_label.to_numpy(), train_files, train_pressures), (test_fp, test_label.to_numpy(), \
                    test_files, test_pressures), train_label, test_label
        
    def hp_opt(self):
        if self.algo == 'nn':
            self.max_batch = 512
            if self.max_batch > self.n_train:
                self.max_batch = self.n_train // 2
            self.space = [(100, 400), #n_units
                    (.0005, .003),#learning rate
                    (2, 15), #patience
                    (12, self.max_batch), #batch size
                    (.01, .6)] #validation split
            
        else:
            self.max_depth_ub = 15
            if self.n_train < 200:
                self.max_depth_ub = 4
            self.space = [(.3, .95), #colsample_bytree
                            (.01, .5),#learning_rate
                            (2, 15), #max_depth
                            (1, 20)] #alpha
            
        hp_df = self.df.sample(frac=self.hp_frac, random_state=self.seed)
        hp_remote_info = [x for x in self.remote_info if x[1] in hp_df['filename'].tolist()]
#         self.hp_instance = HPOpt(hp_df, self.train_pct, self.features, self.property_used, self.target_mean, 
#                             self.target_std, self.seed, hp_remote_info, self.stacked, self.space)
        return HPOpt(hp_df, self.train_pct, self.features, self.property_used, self.target_mean, 
                            self.target_std, self.seed, hp_remote_info, self.stacked, self.space).get_params()
    
    def run_model(self):
        self.params = self.hp_opt()
        self.train_d, self.test_d, self.train_label, self.test_label = self.split()
        self.MODEL = ml.run_model(self.algo, self.train_d, self.n_trees, self.params)
        
        
    def makeResults(self):
        self.run_model()
        if self.algo=='xgb':
            test_predictions = self.MODEL.predict(self.test_d)
            train_predictions = self.MODEL.predict(self.train_d)
            pressures = [35]*(self.n_samples)
            files = self.train_fn + self.test_fn
        if self.algo=='nn':
            test_fp = self.test_d[0]
            train_fp = self.train_d[0]
            files = self.train_d[2] + self.test_d[2]
            pressures = self.train_d[3] + self.test_d[3]
            test_predictions = self.MODEL.predict(test_fp).flatten()
            train_predictions = self.MODEL.predict(train_fp).flatten()        
        
        res_test_predictions, res_test_label, res_train_label, res_train_predictions = ml.unscale(self.property_used, 
                                                                                       test_predictions, 
                                                                                       train_predictions, 
                                                                                       self.test_label, 
                                                                                       self.train_label, 
                                                                                    self.target_mean, 
                                                                                    self.target_std)
        preds = res_train_predictions.tolist() + res_test_predictions.tolist()
        sample_class = ['Train']*len(res_train_predictions) + ['Test']*len(res_test_predictions)
#        truth = res_train_label.tolist() + res_test_label.tolist()
#         results_df = pd.DataFrame({"Filename": files, "Pressure": pressures, "Class": sample_class,
#                                   "Prediction": preds, "Truth": truth})
        results_df = pd.DataFrame({"Filename": files, "Pressure": pressures, "Class": sample_class,
                                  "Prediction": preds})
        return results_df, self.MODEL

In [9]:
#class(HPtrainTestSplit(trainTestSplit):

In [10]:
class HPOpt:
    def __init__(self, df, train_pct, features, property_used, target_mean, target_std, seed, remote_info, stacked, 
                 space, n_trees=50):
        self.df = df
        self.seed = seed
        self.space = space
        self.remote_info = remote_info
        self.stacked = stacked
        self.property_used = property_used
        self.features = features
        self.target_mean = target_mean
        self.target_std = target_std
        self.n_trees = n_trees
        self.N_CALLS = 75
        
        
        if self.stacked:
            self.algo = 'nn'
        else:
            self.algo = 'xgb'
        self.train_pct = train_pct
    
    def objective(self, params):
        MODEL = ml.run_model(self.algo, self.train_d, self.n_trees, params)
        return ml.model_rmse(MODEL, self.train_d, self.test_d, self.stacked, self.algo, self.target_mean, 
                             self.target_std, self.property_used, self.test_label, self.train_label, save=False, 
                             fname=None, subset_inds=None)
    
    def get_params(self):
        HP_TTS = trainTestSplit(self.df, self.train_pct, 
                                                        self.features, self.property_used, self.target_mean,
                                                        self.target_std, self.seed, self.remote_info, 
                                                        self.stacked)
        self.train_d, self.test_d, self.train_label, self.test_label = HP_TTS.split()
        defects = []
        start = time.time()
        r = gp_minimize(self.objective, self.space, n_calls=self.N_CALLS, random_state=self.seed)
        end = time.time()
        self.params = r.x
        return self.params

# Run

In [17]:
global now
now = datetime.datetime.now().strftime("%I_%M%p_on_%B_%d_%Y")
hmofMLdataset('/data/rgur/efrc/ml/results/', 
                   SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data.csv', 
                 SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data.csv',SI_stacked_path=
                '/data/rgur/efrc/prep_data/all_v1/stacked.csv',
                 SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked.csv').makeAllResults()
# hmofMLdataset('/data/rgur/efrc/ml/results/', 
#                    SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data_head.csv', 
#                  SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data_head.csv',SI_stacked_path=
#                 '/data/rgur/efrc/prep_data/all_v1/stacked_head.csv',
#                  SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked_head.csv').makeAllResults()

There are 24 unique feature codes
now is 09_51AM_on_April_17_2020

Starting to Construct Gravimetric Uptake Data Frame
Using start_str_sd CH4_v/v_248_bar
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


/home/modules/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys




Starting To Make Linker Size Columns
Starting to sort Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']

Starting to Construct Isotherm Stacked Data Frame
Using start_str_sd Density
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


/home/modules/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys




Starting To Make Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']




...

Number of trees: 50
Elapsed time during model training:  2.517364740371704
Number of trees: 50
Elapsed time during model training:  1.4805049896240234
Number of trees: 50
Elapsed time during model training:  2.585449695587158
Number of trees: 50
Elapsed time during model training:  2.0450737476348877
Number of trees: 50
Elapsed time during model training:  1.2031035423278809
Number of trees: 50
Elapsed time during model training:  1.6588366031646729
Number of trees: 50
Elapsed time during model training:  1.9607737064361572
Number of trees: 50
Elapsed time during model training:  1.8545923233032227
Number of trees: 50
Elapsed time during model training:  0.9656319618225098
Number of trees: 50
Elapsed time during model training:  0.4258708953857422
Number of trees: 50
Elapsed time during model training:  2.809751510620117
Number of trees: 50
Elapsed time during model training:  1.9791102409362793
Number of trees: 50
Elapsed time during model training:  0.6163744926452637
Number of trees

Elapsed time during model training:  2.8129892349243164
Number of trees: 50
Elapsed time during model training:  0.6247963905334473
Number of trees: 50
Elapsed time during model training:  2.0081443786621094
Number of trees: 50
Elapsed time during model training:  2.7128467559814453
Number of trees: 50
Elapsed time during model training:  0.4242541790008545
Number of trees: 50
Elapsed time during model training:  1.2533166408538818
Number of trees: 50
Elapsed time during model training:  2.839439630508423
Total len of test_df + train_df: 126369
Number of trees: 5000
Elapsed time during model training:  890.1121089458466
Save Results using Fragment grav_code_1100_train_90_seed_10_09_51AM_on_April_17_2020
Total len of test_df + train_df: 12637
Number of trees: 50
Elapsed time during model training:  2.232257127761841
Number of trees: 50
Elapsed time during model training:  2.119675636291504
Number of trees: 50
Elapsed time during model training:  1.7231419086456299
Number of trees: 50
El

/home/modules/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Running code 0100 for gravimetric uptake model
Starting to make KDTree
Finished making KDTree in 452.9501898288727 seconds
Total len of test_df + train_df: 12637
Number of trees: 50
Elapsed time during model training:  1.344712495803833
Number of trees: 50
Elapsed time during model training:  0.5851786136627197
Number of trees: 50
Elapsed time during model training:  2.526928663253784
Number of trees: 50
Elapsed time during model training:  1.309535264968872
Number of trees: 50
Elapsed time during model training:  0.6842174530029297
Number of trees: 50
Elapsed time during model training:  1.3420002460479736
Number of trees: 50
Elapsed time during model training:  2.1672346591949463
Number of trees: 50
Elapsed time during model training:  0.5968196392059326
Number of trees: 50
Elapsed time during model training:  0.993818998336792
Number of trees: 50
Elapsed time during model training:  2.1625986099243164
Number of trees: 50
Elapsed time during model training:  0.6292622089385986
Number

KeyboardInterrupt: 

# Test classes

In [60]:
# obj = hmofMLdataset('/data/rgur/efrc/ml/results/', 
#                    SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data_head.csv', 
#                  SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data_head.csv',SI_stacked_path=
#                 '/data/rgur/efrc/prep_data/all_v1/stacked_head.csv',
#                  SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked_head.csv')
obj = hmofMLdataset('/data/rgur/efrc/ml/results/', 
                   SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data.csv', 
                 SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data.csv',SI_stacked_path=
                '/data/rgur/efrc/prep_data/all_v1/stacked.csv',
                 SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked.csv')

There are 24 unique feature codes
now is 08_42PM_on_April_16_2020


In [61]:
l = obj.makeResults()


Starting to Construct Gravimetric Uptake Data Frame
Using start_str_sd CH4_v/v_248_bar
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


/home/modules/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.




Starting To Make Linker Size Columns
Starting to sort Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']

Starting to Construct Isotherm Stacked Data Frame
Using start_str_sd Density
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


/home/modules/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.




Starting To Make Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']




/home/modules/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Running code 1000 for gravimetric uptake model
Running code 1100 for gravimetric uptake model
Running code 0100 for gravimetric uptake model
Running code 1010 for gravimetric uptake model
Running code 1110 for gravimetric uptake model
Running code 0110 for gravimetric uptake model
Running code 1001 for gravimetric uptake model
Running code 1101 for gravimetric uptake model
Running code 0101 for gravimetric uptake model
Running code 1011 for gravimetric uptake model
Running code 1111 for gravimetric uptake model
Running code 0111 for gravimetric uptake model
Running code 1000 for isotherm model
Running code 1100 for isotherm model
Running code 0100 for isotherm model
Running code 1010 for isotherm model
Running code 1110 for isotherm model
Running code 0110 for isotherm model
Running code 1001 for isotherm model
Running code 1101 for isotherm model
Running code 0101 for isotherm model
Running code 1011 for isotherm model
Running code 1111 for isotherm model
Running code 0111 for isother

In [62]:
FP_DF = l[0] #no geometric fp

In [63]:
type(FP_DF)

__main__.FpDataSet

In [65]:
start = time.time()
FP_DF.run()
end = time.time()
print("Time to finish %s" %end-start)

Starting to make KDTree
Finished making KDTree in 11.281946420669556 seconds
self.train_pct 0.5
self.pct_remote 0.0
self.train_pct 0.5
self.pct_remote 0.0
Total len of test_df + train_df: 12637
Number of trees: 50
Elapsed time during model training:  0.6348891258239746
Number of trees: 50
Elapsed time during model training:  0.3566446304321289
Number of trees: 50
Elapsed time during model training:  0.6056690216064453
Number of trees: 50
Elapsed time during model training:  0.2054276466369629
Number of trees: 50
Elapsed time during model training:  0.12389230728149414
Number of trees: 50
Elapsed time during model training:  0.28655552864074707
Number of trees: 50
Elapsed time during model training:  0.3374826908111572
Number of trees: 50
Elapsed time during model training:  0.11064696311950684
Number of trees: 50
Elapsed time during model training:  0.19299960136413574
Number of trees: 50
Elapsed time during model training:  0.4568748474121094
Number of trees: 50
Elapsed time during mo

Number of trees: 50
Elapsed time during model training:  0.612015962600708
Number of trees: 50
Elapsed time during model training:  1.3808188438415527
Number of trees: 50
Elapsed time during model training:  0.6104238033294678
Number of trees: 50
Elapsed time during model training:  0.6304833889007568
Number of trees: 50
Elapsed time during model training:  0.48162007331848145
Number of trees: 50
Elapsed time during model training:  0.444103479385376
Number of trees: 50
Elapsed time during model training:  0.19505071640014648
Number of trees: 50
Elapsed time during model training:  0.5060126781463623
Number of trees: 50
Elapsed time during model training:  0.7693510055541992
Number of trees: 50
Elapsed time during model training:  0.8559272289276123
Number of trees: 50
Elapsed time during model training:  0.509953498840332
Number of trees: 50
Elapsed time during model training:  0.7662866115570068
Number of trees: 50
Elapsed time during model training:  0.39498186111450195
Number of tr

Number of trees: 50
Elapsed time during model training:  0.7388250827789307
Number of trees: 50
Elapsed time during model training:  0.36505937576293945
Number of trees: 50
Elapsed time during model training:  0.6015791893005371
Number of trees: 50
Elapsed time during model training:  0.5892033576965332
Number of trees: 50
Elapsed time during model training:  0.8857986927032471
Number of trees: 50
Elapsed time during model training:  1.1711337566375732
Number of trees: 50
Elapsed time during model training:  1.1059269905090332
Number of trees: 50
Elapsed time during model training:  1.0684914588928223
Number of trees: 50
Elapsed time during model training:  0.8644540309906006
Number of trees: 50
Elapsed time during model training:  1.0453357696533203
Number of trees: 50
Elapsed time during model training:  0.9896681308746338
Number of trees: 50
Elapsed time during model training:  0.19026827812194824
Number of trees: 50
Elapsed time during model training:  0.43559837341308594
Number of

KeyboardInterrupt: 

In [ ]:
TTS_obj = trainTestSplit(FP_DF.df, .5, FP_DF.features, FP_DF.property_used,
                                                 FP_DF.target_mean, FP_DF.target_std, 0, FP_DF.remote_info,
                                                 FP_DF.stacked)

In [ ]:
df, MODEL = TTS_obj.makeResults()

In [148]:
FP_DF.TTS

[]

In [172]:
TTS_instance = FP_DF.TTS[0]

In [173]:
results_df = TTS_instance.makeResults()

Total len of test_df + train_df: 199
Total len of test_df + train_df: 20
Number of trial parameters 10
Trial parameters [0.876817608501866, 0.4776620138946926, 14, 13]
Size of training set 10
Number of trees: 10
Elapsed time during model training:  0.048571109771728516
Trial parameters [0.7162310961607279, 0.4757213097479468, 11, 2]
Size of training set 10
Number of trees: 10
Elapsed time during model training:  0.033069610595703125
Trial parameters [0.8470231870264011, 0.41011160458203444, 12, 15]
Size of training set 10
Number of trees: 10
Elapsed time during model training:  0.01489877700805664
Trial parameters [0.6103020068460179, 0.43443913824759434, 6, 18]
Size of training set 10
Number of trees: 10
Elapsed time during model training:  0.00978541374206543
Trial parameters [0.3008911318207715, 0.29199009252501223, 8, 6]
Size of training set 10
Number of trees: 10
Elapsed time during model training:  0.008946418762207031
Trial parameters [0.47445916583654557, 0.49368605058809045, 1

# Do preptosplit manually

In [87]:
algo = 'nn'
cat_si_sd = obj.cat_si_sd
SD_ML_DATA_PATH = obj.SD_stacked_path
SI_ML_DATA_PATH = obj.SI_stacked_path
start_str_sd = obj.iso_start_str_sd
end_str_sd = obj.iso_end_str_sd
start_str_si = obj.start_str_si
end_str_si = obj.end_str_si
end_str_sd = obj.iso_end_str_sd
total_frac = 1
del_defective_mofs = obj.del_defective_mofs
add_size_fp = obj.add_size_fp
srt_size_fp = obj.srt_size_fp
size_dependent = None
stacked = True
n_core = 18
del_geometric_fp = False
cat_col_names = obj.cat_col_names
Y_DATA_PATH = obj.Y_DATA_PATH
LS_DICT = obj.LS_dict

In [90]:
if stacked:
    print('\nStarting to Construct Isotherm Stacked Data Frame')
else:
    print('\nStarting to Construct Gravimetric Uptake Data Frame')
print('Using start_str_sd %s' %start_str_sd)
print('Using end_str_sd %s' %end_str_sd)
print('Using start_str_si %s' %start_str_si)
print('Using end_str_si %s' %end_str_si)
print('Total frac equals %s' %total_frac)

if cat_si_sd:
    ml_data_sd = pd.read_csv(SD_ML_DATA_PATH)
    ml_data_si = pd.read_csv(SI_ML_DATA_PATH)

    ml_data_si.columns = [col+'_si' for col in ml_data_si.columns]
    si_cols = ml_data_si.keys().tolist()
    si_start_ind = si_cols.index(start_str_si+'_si')
    si_end_ind = si_cols.index(end_str_si+'_si')
    si_pg_cols = si_cols[si_start_ind + 1:si_end_ind]

    ml_data = ml_data_sd.join(ml_data_si[si_pg_cols], rsuffix='_si')


    sd_cols = ml_data_sd.keys().tolist()
    sd_start_ind = sd_cols.index(start_str_sd)
    sd_end_ind = sd_cols.index(end_str_sd)
    sd_pg_cols = sd_cols[sd_start_ind + 1:sd_end_ind]

    pg_cols = si_pg_cols + sd_pg_cols
else:
    if size_dependent:
        ML_DATA_PATH=SD_ML_DATA_PATH
        start_str = start_str_sd
        end_str= end_str_sd
    else:
        ML_DATA_PATH=SI_ML_DATA_PATH
        start_str = start_str_si
        end_str= end_str_si
    ml_data = load_ml_data(ML_DATA_PATH)
    pg_cols = ml.getPGcolNames(ml_data, start_str, end_str)


non_pg_cols = ml.getNonPGcolNames(add_size_fp, stacked, del_geometric_fp, cat_col_names)

features = pg_cols + non_pg_cols

print('\n')
print("Using following %s features" %len(features))
for i in features:
    print(i)

ml_data = slim(ml_data, total_frac)


Starting to Construct Isotherm Stacked Data Frame
Using start_str_sd Density
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


Using following 844 features
Mafp_Br1_C2_C1_si
Mafp_Br1_C2_C2_si
Mafp_Br1_C2_C3_si
Mafp_Br1_C3_Br1_si
Mafp_Br1_C3_C1_si
Mafp_Br1_C3_C2_si
Mafp_Br1_C3_C3_si
Mafp_Br1_C3_C4_si
Mafp_Br1_C3_N1_si
Mafp_Br1_C3_N2_si
Mafp_Br1_C3_N3_si
Mafp_Br1_C3_O1_si
Mafp_Br1_C4_Br1_si
Mafp_Br1_C4_C2_si
Mafp_Br1_C4_C3_si
Mafp_Br1_C4_C4_si
Mafp_Br1_C4_H1_si
Mafp_Br1_C4_N1_si
Mafp_Br1_C4_N2_si
Mafp_Br1_C4_N3_si
Mafp_Br1_C4_O1_si
Mafp_Br1_C4_O2_si
Mafp_Br1_N2_C2_si
Mafp_Br1_N2_C3_si
Mafp_Br1_N2_C4_si
Mafp_Br1_N2_N1_si
Mafp_Br1_N2_N2_si
Mafp_Br1_N2_N3_si
Mafp_Br1_N3_Br1_si
Mafp_Br1_N3_C2_si
Mafp_Br1_N3_C3_si
Mafp_Br1_N3_H1_si
Mafp_Br1_N3_N2_si
Mafp_Br1_N3_O2_si
Mafp_Br1_O2_C2_si
Mafp_Br1_O2_C3_si
Mafp_Br1_O2_C4_si
Mafp_C1_C2_C2_si
Mafp_C1_C2_C3_si
Mafp_C1_C2_C4_si
Mafp_C1_C2_F1_si
Mafp_C1_C2_H1_si
Mafp_C1_C2_O1_si
Mafp_

NameError: name 'slim' is not defined

In [98]:
if add_size_fp:
    size_names = ['size_%s' %n for n in range(20)] 
    print('Starting To Make Linker Size Columns')        
    if LS_DICT == None:
        for col in [col for col in ml_data.keys() if 'Smiles' in col]:
            makeSizeCol(ml_data, col)
        if srt_size_fp:
            print('Starting to sort Linker Size Columns')           
            a = ml_data[size_names].values
            a.sort(axis=1)
            a = a[:, ::-1]
            ml_data[size_names] = a
    else:
        smiles_cols = [col for col in ml_data.keys() if 'Smiles' in col]
        
        def makeLSfromDict(row):
            fail = 0
            f_name = row['filename']
            try:
                LS_list = LS_DICT[f_name].tolist()
            except:
                fail += 1
                smiles = row[smiles_cols]
                LS_list = [ml.getNAtoms(s) for s in smiles]
            print(fail)
            return pd.Series(row.tolist() + LS_list)

        old_cols = ml_data.columns.tolist()
        ml_data = ml_data.apply(makeLSfromDict, axis=1)
        ml_data.columns = old_cols + size_names
    print('Finished Making Linker Size Columns')

Starting To Make Linker Size Columns
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Finished Making Linker Size Columns


In [95]:
ml_data['size_0']

0      42
1      38
2      12
3      30
4      94
5      10
6      28
7      44
8      30
9      24
10     36
11     32
12     40
13     12
14     62
15     49
16     16
17     39
18     12
19     30
20     26
21     40
22     72
23     64
24     37
25     16
26     28
27     50
28     30
29     73
       ..
169    26
170    10
171     8
172    30
173    14
174    48
175    10
176    44
177    18
178    29
179    41
180    32
181    26
182    30
183    28
184    47
185    10
186    32
187    50
188    30
189    26
190    19
191    14
192    14
193    16
194    20
195    16
196     8
197    24
198    16
Name: size_0, Length: 199, dtype: int64